In [1]:
import pandas as pd
import numpy as np
from difflib import get_close_matches

from linestar import get_historical_data

In [2]:
def close_matches(x, possible):
    matches = get_close_matches(x, possible)
    if matches:
        return matches[0]
    else:
        return np.nan

In [3]:
players = get_historical_data("2022-05-17")

ownership = pd.read_csv("2022-05-17.csv")
ownership = ownership.dropna()
ownership["Actual Ownership"] = (ownership["Actual Ownership"].str[:-1].astype(float)) / 100
ownership = ownership[["Player", "Actual Ownership"]]
# Ownership data names don't match exactly to the players, so get close matches
ownership["Player"] = ownership["Player"].apply(lambda x: close_matches(x, players["Player"]))

data = players.merge(ownership, on="Player")

teams = pd.get_dummies(data["Team"])
games = pd.get_dummies(data["Game"])

/home/eadains/DFS/linestar.py:51: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 51 of the file /home/eadains/DFS/linestar.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)


In [1]:
class OpponentTeams:
    def __init__(self, players):
        self.players = players
        self.position_nums = {"P": 1, "C/1B": 1, "2B": 1, "3B": 1, "SS": 1, "OF": 3, "UTIL": 1}
        self.pos_mat = self.get_position_matrix()
        self.teams = pd.get_dummies(players["Team"])
        self.games = pd.get_dummies(players["Game"])
        
    def get_position_matrix(self):
        """
        Get matrix of 0/1 values indicating whether a player can fill a particular postions
        Each row is a player and each column is a position
        """
        positions = pd.get_dummies(self.players["Position"])
        # If a player plays either C or 1B, they can fill the 1B/C position
        positions["C/1B"] = (positions["1B"].astype(bool) | positions["C"].astype(bool)).astype(int)
        positions = positions.drop(columns=["1B", "C"])
        # Reorder columns
        positions = positions[["P", "C/1B", "2B", "3B", "SS", "OF"]]
        return positions
    
    def position_choice(self, position):
        """
        Chooses a player randomly for given position
        Returns numpy array containing player names selected for position
        """
        # If position is C/1B cathers or first-basemen can fill
        if position == "C/1B":
            data = self.players.loc[self.players["Position"].isin(["C", "1B"]), "Player"]
            probs = self.players.loc[self.players["Position"].isin(["C", "1B"]), "Actual Ownership"]
        # If position is UTIL then anyone except pitchers can fill
        elif position == "UTIL":
            data = self.players.loc[self.players["Position"] != "P", "Player"]
            probs = self.players.loc[self.players["Position"] != "P", "Actual Ownership"]
        else:
            data = self.players.loc[self.players["Position"] == position, "Player"]
            probs = self.players.loc[self.players["Position"] == position, "Actual Ownership"]
        # Set probability of choosing each player to the actual ownership numbers normalized
        # to sum to 1
        probs = probs / probs.sum()
        choice = np.random.choice(data, self.position_nums[position], replace=False, p=probs)
        return choice
    
    def select_team(self):
        """
        Select entire team
        Returns boolean integer array
        """
        team = []
        for position in self.position_nums.keys():
            team.append(self.position_choice(position))
        team = np.concatenate(team)
        return self.players["Player"].isin(team).astype(int)
    
    def check_valid_team(self, x):
        """
        Check if a team is valid given FanDuel roster rules
        """
        salary = x @ self.players["Salary"] <= 35000
        min_salary = x @ self.players["Salary"] >= 34000
        teams_con = (x @ self.teams >= 1).sum() >= 3
        games_con = (x @ self.games >= 1).sum() >= 2
        players_con = ((x * (~self.pos_mat["P"].astype(bool))) @ teams <= 4).all()
        total = np.sum(x) == 9
        positions_max = [1, 2, 2, 2, 2, 4]
        positions_min = [1, 1, 1, 1, 1, 3]
        positions_max_con = (x @ self.pos_mat <= positions_max).all()
        positions_min_con = (x @ self.pos_mat >= positions_min).all()
        return salary & teams_con & games_con & players_con & total & positions_max_con & positions_min_con
    
    def get_team(self):
        while True:
            team = self.select_team()
            if self.check_valid_team(team):
                return team
            else:
                continue
                
    def get_order_stat(self, points, cutoff, num_opp):
        opp_teams_scores = [self.get_team() @ points for x in range(num_opp)]
        return opp_teams_scores[cutoff-1]

In [132]:
opps = OpponentTeams(data)

In [135]:
opps.get_team()

0     0
1     0
2     1
3     0
4     0
     ..
74    0
75    0
76    0
77    0
78    0
Name: Player, Length: 79, dtype: int64